# Yemeksepeti Data Collection

This notebook aims to employ various web scraping techniques to gather data on restaurants from the popular platform "Yemeksepeti". The collected data will be stored in the JSON format within the designated directory named `collected_data`.

The dataset will encompass a wide range of information pertaining to the restaurants, encompassing details such as their menu items, pricing, customer comments, ratings, and corresponding timestamps.

Subsequently, the primary goal is to conduct semantic analysis on the acquired dataset, with the aim of gaining deeper insights into consumer behavior patterns within the context of Turkey.

In [ ]:
# import libraries

import requests

## A List of Resturants in Istanbul

In this section, our goal is to gather a comprehensive list of all restaurants in Istanbul, complete with detailed information such as their address, budget, cuisine offerings, and most importantly, the URLs leading to their respective pages on Yemeksepeti.

To accomplish this task, we will send a specific request to Yemeksepeti's database and parse the response into a JSON format. This will enable us to efficiently organize and explore the gathered information, providing valuable insights

In [22]:
# Yemeksepeti uses the Delivery Hero servers to store it's data, hence we can get resturant's data by sending a request to delivery hero
# the arguments ...&offset=0&limit=500&... can be added to the url to specify the data request size
yemeksepeti_request_url = "https://disco.deliveryhero.io/listing/api/v1/pandora/vendors?language_id=3&vertical=restaurants&country=tr&include=characteristics&configuration=Variantn&sort=&city_id=34"

# the headers are acquired from the requests from yemeksepeti's webpage
headers = {
'accept': 'application/json, text/plain, */*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-US,en;q=0.9',
'dnt': '1',
'origin': 'https://www.yemeksepeti.com',
'perseus-client-id': '1683402284518.770138949646337300.d4t6e4b8h2',
'perseus-session-id': '1683746387414.963635505715628500.nu2sj79e0c',
'referer': 'https://www.yemeksepeti.com/',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
'x-disco-client-id': 'web',
'x-fp-api-key': 'volo '
}

# send the request and store the response
print("sending a request...")
response = requests.get(yemeksepeti_request_url, headers=headers)

# save the response data as json
print("saving the json data...")
data_export_path = '../collected_data/yemeksepeti_resturants_data.json'
with open(data_export_path, 'wb') as of:
    of.write(response.content)

print(f"succefully exported the data. path: `{data_export_path}`")

sending a request...
saving the json data...
succefully exported the data. path: `../collected_data/yemeksepeti_resturants_data.json`
